 # 3. Trénink modelu predikce prokliku
V následujícím notebooku se pokusíme natrénovat model predikce prokliku pomocí knihovny [vowpal wabbit](https://vowpalwabbit.org/).

In [ ]:
import os

try:
    from google.colab import drive

    drive.mount('/content/gdrive')
    BASE_DIR = "/content/gdrive/MyDrive/itacademy2022"
    IN_COLAB = True
except:
    IN_COLAB = False
    BASE_DIR = ".."

MIND_DATA_SOURCE_DIR = "tmp/mind"
ORIGINAL_TRAIN_INPUT_DIR = os.path.join(BASE_DIR, MIND_DATA_SOURCE_DIR, "train/")
ORIGINAL_TEST_INPUT_DIR = os.path.join(BASE_DIR, MIND_DATA_SOURCE_DIR, "test/")
OUTPUT_DIR = os.path.join(BASE_DIR, "output")

### Vowpal wabbit
Vowpal wabbit (vw) je extrémně rychlá implementace logistické regrese napsaná v c++. Má celou řadu užitečných vlastností, díky kterým se osvědčila v produkčních prostředích nejen Seznamu.

Pro účely tohoto workshopu budeme využívat [sklearnový wrapper](https://vowpalwabbit.org/docs/vowpal_wabbit/python/latest/reference/vowpalwabbit.sklearn.html), abychom mohli s vw jednoduše interagovat v rámcí notebooku.

In [ ]:
!pip install vowpalwabbit

 # Připrava datasetu
Pro trénování a vyhodnocení modelu potřebujeme tři typy datasetů:
 - trénovací dataset: slouží přímo k vytvoření modelu 
 - validační dataset: slouží k porovnání více modelů mezi sebou např. pro vyhodnocení optimálních hyper parametrů
 - testovací dataset: slouží pro odhad výkonu modelu v produkci

Více se lze dozvědet v následujícím [článku](https://machinelearningmastery.com/difference-test-validation-datasets/).

Pro účely workshopu jsme zvolili menší MIND dataset, který bohužel obsahuje pouze trénovací a testovací dataset, jeden nám tedy chybí a z tohoto důvodu jsme se rozhodli rozdělit testovací dataset na dvě poloviny a první využít jako validační dataset a druhý jako testovací dataset, což není standardní (měli bychom dělit trénovací dataset), ale vzhledem k výrazným rozdílum mezi trénovacím a testovacím datasetem nám to umožní lépe modelovat data v rámci našeho cvičení.

Formát našeho datasetu bude následující:
 - historie přečtených článku uživatele
 - historie kategorií, do kterých spadali články z uživatelovi historie
 - historie subkategorií, do kterých spadali články z uživatelovi historie
 - historie titulků, které měli články z uživatelovi historie
 - predikování článek a nebo také imprese
 - kategorie imprese
 - subktegorie imprese
 - titulek imprese
 - informace, zda-li uživatel na daný článek kliknul

Nejprve vytvoříme dataset pomocí knihovny pandas a funkce `prepare_dataset_pd`. Následně data transformuje do [formátu vstupních dat](https://github.com/VowpalWabbit/vowpal_wabbit/wiki/Input-format), jež vyžaduje vw.


In [ ]:
# import necessary functionality
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import gc
from vowpalwabbit.sklearn import VWClassifier, VW
from sklearn import metrics
from tqdm.notebook import tqdm
from sklearn.feature_extraction. _stop_words import ENGLISH_STOP_WORDS
from sklearn.model_selection import train_test_split

In [ ]:
RANDOM_SEED = 42

def get_vocabulary(news, min_occ = 10):
    word_occ = news.title.map(preprocess_title).str.split(" ").explode().value_counts()
    vocabulary = set(word_occ[word_occ > min_occ].index.tolist()) - ENGLISH_STOP_WORDS
    return vocabulary


def preprocess_title(title, vocabulary=None):
    words = set(re.findall('\w{3,}', title.lower()))
    if vocabulary:
        return ' '.join(words & vocabulary)
    else:
        return ' '.join(words)


def prepare_dataset_pd(behaviors,
                       news,
                       only_category=None,
                       shuffle=False,
                       random_seed=42,
                       vocabulary=None):
    news_fildered = news

    if only_category is not None:
        news_filtered = news[news.category == only_category]
    else:
        news_filtered = news
    news_filtered["title_proc"] = news_filtered["title"].map(lambda t: preprocess_title(t, vocabulary))

    behaviors_histories_only = (
        behaviors[[
            "slateid", "history"
        ]].assign(history=lambda x: x["history"].fillna("").str.split()).
        explode("history").reset_index(drop=True).reset_index(
            drop=False)  # trick to preserve original ordering through merge
        .merge(
            news_filtered[["category", "subcategory", "title_proc"]],
            left_on="history",
            right_index=True,
            how="inner",
            sort=False,
        ).sort_values("index").drop("index",
                                    axis=1)  # restore original ordering
        .groupby("slateid", as_index=False).agg({
            "history": lambda x: x.unique().tolist(),
            "category": lambda x: x.unique().tolist(),
            "subcategory": lambda x: x.unique().tolist(),
            "title_proc": lambda x: x.map(lambda xx: xx.split()).apply(pd.Series).unstack().dropna().unique().tolist()
        }).assign(history=lambda x: x["history"].str.join(" "),
                  category=lambda x: x["category"].str.join(" "),
                  subcategory=lambda x: x["subcategory"].str.join(" "),
                  title=lambda x: x["title_proc"].str.join(" ")))

    # filter impressions to news-only
    behaviors_impressions_only = (
        behaviors[[
            "slateid", "impressions"
        ]].assign(impressions=lambda x: x["impressions"].fillna("").str.split(
        )).explode("impressions").assign(
            impression_id=lambda x: x["impressions"].str.split("-").str[0]).
        assign(click=lambda x: x["impressions"].str.split(
            "-").str[1].astype(int)).reset_index(drop=True).reset_index(
                drop=False)  # trick to preserve original ordering through merge
        .merge(news_filtered[["category", "subcategory", "title_proc"]],
               left_on="impression_id",
               right_index=True,
               how="inner",
               sort=False).sort_values("index").drop(["index", "impressions"],
                                                     axis=1).rename(columns={"title_proc": "title"}))

    df = behaviors_impressions_only.merge(
        behaviors_histories_only,
        on="slateid",
        how="inner",
        suffixes=["_i", ""]).assign(history=lambda x: x.history.fillna(" "))

    if shuffle:
        df = df.sample(frac=1.0, random_state=random_seed)

    return df


def prepare_dataset_vw(
    df,
    output_path,
    cls_weights=None,
    include_categories=False,
    include_subcategories=False,
    include_title=False
):
    with open(output_path, 'w') as f:
        for _, row in df.iterrows():
            # write label and sample weight
            sample_weight = f"{cls_weights[row.click]:0.3f}" if cls_weights else '1'
            f.write(f"{1 if int(row.click) else -1} {sample_weight} ")
            # write user history
            f.write(
                f"|h {row.history} |i {row.impression_id}")

            if include_categories:
                f.write(f"|c {row.category} |j {row.category_i}")

            if include_subcategories:
                f.write(f"|k {row.subcategory} |l {row.subcategory_i}")

            if include_title:
                f.write(
                    f"|t {row.title} |o {row.title_i}"
                )

            f.write("\n")

In [ ]:
behaviors_train = pd.read_csv(
    os.path.join(ORIGINAL_TRAIN_INPUT_DIR, "behaviors.tsv"),
    sep="\t",
    names=["slateid", "userid", "time", "history", "impressions"]
).sample(frac=0.3, random_state=RANDOM_SEED)

In [ ]:
behaviors_val_test = pd.read_csv(
    os.path.join(ORIGINAL_TEST_INPUT_DIR, "behaviors.tsv"),
    sep="\t",
    names=["slateid", "userid", "time", "history", "impressions"]
).sample(frac=0.3, random_state=RANDOM_SEED)

In [ ]:
news_train = pd.read_csv(
    os.path.join(ORIGINAL_TRAIN_INPUT_DIR, "news.tsv"),
    sep="\t",
    names=["newsid", "category", "subcategory", "title", "abstract", "url", "title_entities", "abstract_entities"]
).set_index("newsid")

In [ ]:
vocabulary = get_vocabulary(news_train)

In [ ]:
news_val_test = pd.read_csv(
    os.path.join(ORIGINAL_TEST_INPUT_DIR, "news.tsv"),
    sep="\t",
    names=["newsid", "category", "subcategory", "title", "abstract", "url", "title_entities", "abstract_entities"]
).set_index("newsid")

In [ ]:
behaviors_train_ex = prepare_dataset_pd(
    behaviors_train,
    news_train,
    shuffle=True,
    vocabulary=vocabulary
)

In [ ]:
behaviors_test, behaviors_val = train_test_split(
    behaviors_val_test,
    test_size=0.5
)

In [ ]:
behaviors_val_ex = prepare_dataset_pd(
    behaviors_val,
    news_val_test,
    vocabulary=vocabulary
)

In [ ]:
behaviors_test_ex = prepare_dataset_pd(
    behaviors_test,
    news_val_test,
    vocabulary=vocabulary
)

 ## Jak pracovat s nevyváženým datasetem?
Náš dataset je silně nevyvážený ve prospěch negativních příkladu, které více než 10x prevyšují pozitivní příklady, jak takovouto situaci můžeme řešit:
 - oversampling/undersampling minoritní/majoritní třídy
 - vážením klasifikačních tříd
 - vážením trénovacích příkladu

VW podporuje [poslední možnost řešení](https://github.com/VowpalWabbit/vowpal_wabbit/wiki/Input-format#simple).

V následujících buňkách vypočteme váhy pozitivních a negativních příkladu.

In [ ]:
# compute positive & negative example counts
neg_count, pos_count = behaviors_train_ex.click.value_counts()

In [ ]:
cls_weights = {
    0:  (1 / neg_count) * ((neg_count + pos_count) / 2.0),
    1:  (1 / pos_count) * ((neg_count + pos_count) / 2.0)
}

In [ ]:
prepare_dataset_vw(
    behaviors_train_ex,
    "/tmp/vw_train_data_all.dat",
    include_categories=True,
    include_subcategories=True,
    include_title=True,
    cls_weights=cls_weights
)

In [ ]:
prepare_dataset_vw(
    behaviors_val_ex,
    "/tmp/vw_val_data_all.dat",
    include_categories=True,
    include_subcategories=True,
    include_title=True
)

In [ ]:
prepare_dataset_vw(
    behaviors_test_ex,
    "/tmp/vw_test_data_all.dat",
    include_categories=True,
    include_subcategories=True,
    include_title=True
)

# Kontrola vytvorenych datasetu

In [ ]:
!du -sh "/tmp/vw_train_data_all.dat"
!wc -l "/tmp/vw_train_data_all.dat"
!head -n 5 "/tmp/vw_train_data_all.dat"

In [ ]:
!du -sh "/tmp/vw_val_data_all.dat"
!wc -l "/tmp/vw_val_data_all.dat"
!head -n 5 "/tmp/vw_val_data_all.dat"

In [ ]:
!du -sh "/tmp/vw_test_data_all.dat"
!wc -l "/tmp/vw_test_data_all.dat"
!head -n 5 "/tmp/vw_test_data_all.dat"

# Trénování modelu

Pro experimentování budou důležité následující základní parametry:
 - convert_to_vw: parametr určující zda-li se má provést konverze vstupu do formátu vw
 - convert_labels: transformace anotaci z rozsahu [0,1] do rozsahu [-1,1]
 - loss_function: loss funkce - v našem případě to bude hodnota 'logistic'
 - link: opět hodnota 'logistic' - zajistí, že budeme predikovat hodnoty z rozsahu [0, 1] - tedy z [bernoulliho distribuce](https://en.wikipedia.org/wiki/Bernoulli_distribution)
 - quiet: specifikuje, zda-li chceme na výstupu debug informace 
 - keep: omezuje množství používaných signálu z datasetu
 - passes: specifikuje množství průchodu datasetem při trénování
 - bit_precision: definuje velikost hashovacího prostoru
 - holdout_off: specifikuje, zda-li použít část validačních dat pro cross validace
 - kill_cache: specifikuje, zda-li chceme pokaždé začít s prazdnou cache
 - data: specifikuje cestu k trénovacím datům
 - cache_file: cesta ke cache soubor
 - random_seed: seed pro inicializace modelu, zajistí reprodukovatelnost
 - hash_seed: seed pro hashování signalu, zajistí reprodukovatelnost

V rámcí workshopu si nejprve demonstrujeme jak natrénovat základní popularity model a následně si sami vyzkoušíte modelování a budete se pokoušet dosáhnout co možná nejlepšího skóré.

## Metriky
Stanovení vhodných metrik pro vyhodnocení modelovací úlohy je kritické.

V oblasti doporučování se využíváji metriky zaměřené na zhodnocení celého doporučovaného slatu (aneb většinou prezentujeme uživateli více než jeden konkrétní objekt):
 - [NDCG](https://en.wikipedia.org/wiki/Discounted_cumulative_gain)
 - [recall@k](https://medium.com/@m_n_malaeb/recall-and-precision-at-k-for-recommender-systems-618483226c54)
 - [precission@k](https://medium.com/@m_n_malaeb/recall-and-precision-at-k-for-recommender-systems-618483226c54)
 - [novelty](https://medium.com/@ayanglaishram/novelty-in-recommender-system-1a3da04e3b1f)
 - ...

V rámci workshopu budeme pracovat s metrikami zaměřenými na konkrétní článek, které budou zhodnocovat jak dobře jsme odhadli relativní skóre mezi články (pří doporučování článku je uživateli předkládáme od největšího skóre):
  - [AUC](https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5)
  - [Log loss](https://en.wikipedia.org/wiki/Cross_entropy#Cross-entropy_loss_function_and_logistic_regression)


In [ ]:
# define variable for storing results
MODEL_RESULTS = []

In [ ]:
# define common parameters for vw training
base_params = {
    "convert_to_vw": False,
    "convert_labels": False,
    "loss_function": "logistic",
    "link": "logistic",
    "quiet": True,
    "keep": "i",
    "passes": 1,
    "bit_precision": 22,
    "holdout_off": True,
    "kill_cache": True,
    "data": "/tmp/vw_train_data_all.dat",
    "cache_file": "/tmp/vw_cache.out",
    "random_seed": RANDOM_SEED,
    "hash_seed": RANDOM_SEED
}

In [ ]:
# load validation data for vw evaluation
with open("/tmp/vw_val_data_all.dat", "r") as f:
    val_data_wv_raw_all = f.read()

val_data_wv_all = [r for r in val_data_wv_raw_all.split("\n") if r]

In [ ]:
def train_eval_vw(base_params, params, vw_val_data, val_data_df):
    global MODEL_RESULTS

    for param in tqdm(params):
        result = {}
        a_params = {**base_params, **param}
        vw_classifier = VWClassifier(**a_params)
        vw_classifier.fit()
        evaluation = evaluate_model(vw_classifier, val_data_df, vw_val_data)

        MODEL_RESULTS.append({
            "all_params": a_params,
            "params": param,
            "model": vw_classifier,
            **evaluation
        })


def evaluate_model(model, data_df, vw_data):
    result = {}
    result["predictions"] = model.predict_proba(vw_data)
    result["labels"] = data_df.click.tolist()
    result["auc"] = metrics.roc_auc_score(
       data_df.click.tolist(),
       result["predictions"][:, 1]
    )
    result["log_loss"] = metrics.log_loss(
        data_df.click.tolist(),
        result["predictions"][:, 1]
    )
    result["roc_curve"] = metrics.roc_curve(
        data_df.click.astype(int).tolist(),
        result["predictions"][:, 1]
    )

    return result


def visualize_evaluation(result):
    print(f"Model configuration: {result['params']}")
    print(f"Log loss: {result['log_loss']}")

    plt.plot(result["roc_curve"][0], result["roc_curve"][1])
    plt.plot([0, 1], [0, 1])
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.title("test AUC = %f" % (result['auc']))
    plt.axis([-0.05, 1.05, -0.05, 1.05])

    plt.show()

    y_pred_class = result["predictions"][:, 1] > 0.5
    cm = metrics.confusion_matrix(result["labels"], y_pred_class)
    disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()


def pick_best_result():
    global MODEL_RESULTS

    if not MODEL_RESULTS:
        return None

    best_result = MODEL_RESULTS[0]

    for r in MODEL_RESULTS[1:]:
        if r["auc"] > best_result["auc"]:
            best_result = r
    
    return best_result

In [ ]:
train_eval_vw(base_params, [{}], val_data_wv_all, behaviors_val_ex)

In [ ]:
visualize_evaluation(MODEL_RESULTS[-1])

# Dokážete natrénovat lepší model?
Není to tak jednoduché jak by se zdálo! Zmíněný popularity model představuje docela silnou baseline a není jednoduché ji překonat.

Tento dataset obsahuje skutečná produkční data a reálné problémy, se kterými se potkáte v produkčním prostředí. Konkrétně v oblasti doporučování zpravodajských článku narazíte na tzv. item cold-start problém, více si o tomto fenoménu můžete nastudovat [tady](https://en.wikipedia.org/wiki/Cold_start_(recommender_systems)#New_item).

K item cold-startu dochází v důsledků toho, že každý den vznikají nové unikatní članky (zprávy), které ještě ani žádný uživatel neviděl a tím pádem je neviděl ani váš model v rámci trénovacích dat. Tento problém lze v produkčním systému mitigovat např. pomoci využítí techniky [contextového bandity](https://vowpalwabbit.org/docs/vowpal_wabbit/python/latest/tutorials/python_Contextual_bandits_and_Vowpal_Wabbit.html#contextual-bandits). My se však budeme zaměřovat na jiný typ řešení a sice přes vhodnou reprezentaci doporučovaných článku, přičemž dataset obsahuje následující metadata o článcích:
 - titulek článku(t) a imprese(o)
 - kategorie článku(c) a imprese(j)
 - subcategorie článku(k) a imprese(l)

Pro účely experimentování s výkonem modelu se můžou hodit tyto parametry:
  - passes: určuje kolik průchodu se má využít pro trénovaní
  - keep: určuje jaké signály se mají použít pro trénování, tzn. umožnuje využít pouze subset signálu z datasetu
  - interactions: umožnuje zkombinovat více signálu pro zvýšení komplexity modelu
  - quadratic: podobně jako parametr 'interactions', ale pouze kvadratické kombinace
  - cubic: podobně jako parametr 'quadratic', ale pouze kubické kombinace
  - bit precission: určuje velikost prostoru všech signálu
  - l2: zapne l2 regularizaci
  - l1: zapne l1 regularizaci
  - learning_rate: nastaví koeficient ovlivňující velikost gradientů
  - ignore_linear: odstranění jednoduché kombinace signálu

**!!!Pokud experimentujete s novým signálem a jeho kombinací, nezapomeňte ho úvest taky v rámci parametru 'keep'!!!**

[Hyperparametr optimalizace](https://en.wikipedia.org/wiki/Hyperparameter_optimization) je process, při kterém se snažíme minimalizovat chybu na validačním datasetu, většinou je plně automatizovaný. V případě vw lze využít [vw-hypersearch](https://github.com/VowpalWabbit/vowpal_wabbit/blob/master/utl/vw-hypersearch). Sklearn taktéž nabízí funkcionalitu pro hledání [optimalizaci hyperparametru](https://scikit-learn.org/stable/modules/grid_search.html). V rámci workshopu si tento process vyzkoušíme manuálně.



In [ ]:
## example of feature interactions
# train_eval_vw(base_params, [{
#     "keep": "hi",
#     "interactions": "hi",
#     "passes": 2
# }], val_data_wv_all, behaviors_val_ex)

# visualize_evaluation(MODEL_RESULTS[-1])

In [ ]:
train_eval_vw(base_params, [{
    "keep": "hi",
    "interactions": "hi",
    "passes": 2
}], val_data_wv_all, behaviors_val_ex)

visualize_evaluation(MODEL_RESULTS[-1])

# Finalni evaluace modelu
Pro odhad výkonu modelu v produkci využijeme testovací dataset, který využijeme pouze jednou pro finálni evaluaci modelu.



In [ ]:
with open("/tmp/vw_test_data_all.dat", "r") as f:
    test_data_wv_raw_all = f.read()

test_data_wv_all = [r for r in test_data_wv_raw_all.split("\n") if r]

In [ ]:
# uncomment in order to attempt final evaluation
# best_result = pick_best_result()
# test_evaluation = evaluate_model(best_result["model"], behaviors_test_ex, test_data_wv_all)
# visualize_evaluation({**best_result, **test_evaluation})

# Závěrem
Doufáme, že Vás modelování bavilo. Pokud byste chtěli Vaše modely srovnat se světovou špičkou, bude potřeba natrénovat model na [plné datové sadě](https://msnews.github.io/#getting-start).

Pokud by Vás zajímaly modely a signály, které na této datové sadě uspěly, tak lze nalézt informace [tady](https://paperswithcode.com/sota/news-recommendation-on-mind), [tady](https://msnews.github.io/assets/doc/ACL2020_MIND.pdf) a [tady](https://msnews.github.io/assets/doc/1.pdf).

## Na co jsme zapoměli?
V rámci modelace jsme nezkoumali [fungování modelu](https://medium.com/ing-blog/model-explainability-how-to-choose-the-right-tool-6c5eabd1a46a) (konkrétně například významovost jednotlivých vah) a ani jsme se nedívali na vyprodukované predikce. Pochopení fungování modelu je v praxi enormně důležité a typicky se volí model, jemuž jsme schopni poruzumět oproti modelu, který sice na metrikách funguje brilantně, ale není zřéjmě proč.

V případě vw není úplně jednoduché k této informaci dospět a je potřeba vyvinout trochu snahy, je potřeba využít [audit](https://vowpalwabbit.org/docs/vowpal_wabbit/python/latest/tutorials/cmd_linear_regression.html) parametr. 